In [465]:
import numpy as np
import pandas as pd
from queue import Queue
import networkx as nx
import time
from PIL import Image

In [466]:
grid_original = np.asarray(pd.read_csv('./inicial_easy.csv').iloc[:, 1:])

grid_original.shape

(0, 0)

In [ ]:
with open('./inicial_easy.csv', 'r') as file:
    content = file.read().strip()

# Interpretar el contenido del archivo
left_side, right_side, boat_position = content.split(';')

# Contar el número de ovejas y lobos en el lado izquierdo
ovejas_izq = left_side.count('O')
lobos_izq = left_side.count('L')

# Contar el número de ovejas y lobos en el lado derecho
ovejas_der = right_side.count('O')
lobos_der = right_side.count('L')

# Determinar la posición de la barca
barca_pos = 'izq' if boat_position.strip() == 'I' else 'der'

initial_state = (ovejas_izq, lobos_izq, ovejas_der, lobos_der, barca_pos)
print("Estado inicial:", initial_state)

Estado inicial: (2, 2, 0, 0, 'izq')


In [ ]:
initial_state = (ovejas_izq, lobos_izq, ovejas_der, lobos_der, barca_pos)
objective_state = (0, 0, ovejas_izq, lobos_izq, 'der')

In [470]:
operations = [
    (1, 0),  # Mover 1 oveja
    (0, 1),  # Mover 1 lobo
    (1, 1),  # Mover 1 oveja y 1 lobo
    (2, 0),  # Mover 2 ovejas
    (0, 2)   # Mover 2 lobos
]

In [471]:
class Node:
    def __init__(self, value, parent=None):
        self.value = value
        self.children = []
        self.parent = parent

    def add_child(self, child):
        node = Node(child, parent=self)
        self.children.append(node)
        return node

class Tree:
    def __init__(self, root):
        self.root = root
         
def find_path_to_root(objective_node):
    path = []
    current_node = objective_node
    while current_node is not None:
        path.insert(0, current_node.value)
        current_node = current_node.parent
    return path

In [472]:
def is_valid_state(ovejas_izq, lobos_izq, ovejas_der, lobos_der):
    if (ovejas_izq < 0 or lobos_izq < 0 or ovejas_der < 0 or lobos_der < 0):
        return False
    if (ovejas_izq > 0 and lobos_izq > ovejas_izq):
        return False
    if (ovejas_der > 0 and lobos_der > ovejas_der):
        return False
    return True

In [ ]:
def generate_children(current_state, operations):
    ovejas_izq, lobos_izq, ovejas_der, lobos_der, barca_pos = current_state
    children = []
    
    for op in operations:
        if barca_pos == 'izq':
            # Mover animales del lado izquierdo al derecho
            new_ovejas_izq = ovejas_izq - op[0]
            new_lobos_izq = lobos_izq - op[1]
            new_ovejas_der = ovejas_der + op[0]
            new_lobos_der = lobos_der + op[1]
            new_barca_pos = 'der'
        else:
            # Mover animales del lado derecho al izquierdo
            new_ovejas_izq = ovejas_izq + op[0]
            new_lobos_izq = lobos_izq + op[1]
            new_ovejas_der = ovejas_der - op[0]
            new_lobos_der = lobos_der - op[1]
            new_barca_pos = 'izq'
        
        new_state = (new_ovejas_izq, new_lobos_izq, new_ovejas_der, new_lobos_der, new_barca_pos)
        
        # Verificar si el nuevo estado es válido
        if is_valid_state(new_ovejas_izq, new_lobos_izq, new_ovejas_der, new_lobos_der): 
            children.append(new_state)
    
    return children


In [ ]:
def construct_solution_BFS(root, objective_state):
    tree = Tree(root)
    nodes_to_expand = Queue()
    nodes_to_expand.put(root)
    visited_nodes = set()
    visited_nodes.add(root.value)

    found = False
    path = []

    while not nodes_to_expand.empty():
        current_node = nodes_to_expand.get()
        current_state = current_node.value

        if current_state == objective_state:
            found = True
            path = find_path_to_root(current_node)
            break

        children = generate_children(current_state, operations)
        for child_state in children:
            if child_state not in visited_nodes:
                child_node = current_node.add_child(child_state)
                nodes_to_expand.put(child_node)
                visited_nodes.add(child_state)

    return tree, path, visited_nodes


In [478]:
start_time = time.time()
root = Tree(Node(initial_state))  
tree, solution_path, visited_nodes = construct_solution_BFS(root.root, objective_state)  
end_time = time.time()

# Mostrar la solución y el tiempo de ejecución
print("Solución:", solution_path)
print("Tiempo de ejecución:", end_time - start_time, "segundos")

Solución: [(2, 2, 0, 0, 'izq'), (1, 1, 1, 1, 'der'), (2, 1, 0, 1, 'izq'), (0, 1, 2, 1, 'der'), (1, 1, 1, 1, 'izq'), (0, 0, 2, 2, 'der')]
Tiempo de ejecución: 0.0010178089141845703 segundos


In [ ]:
import heapq
import time


def heuristic(state):
    ovejas_izq, lobos_izq, ovejas_der, lobos_der, _ = state
    return ovejas_izq + lobos_izq

def define_operations():
    return [
        (1, 0),
        (0, 1),
        (2, 0),
        (0, 2),
        (1, 1),
    ]

def generate_children(state, operations):
    ovejas_izq, lobos_izq, ovejas_der, lobos_der, barca_pos = state
    children = []
    
    for op in operations:
        ovejas_move, lobos_move = op
        
        if barca_pos == 'izq':
            new_ovejas_izq = ovejas_izq - ovejas_move
            new_lobos_izq = lobos_izq - lobos_move
            new_ovejas_der = ovejas_der + ovejas_move
            new_lobos_der = lobos_der + lobos_move
            new_barca_pos = 'der'
        else:
            new_ovejas_izq = ovejas_izq + ovejas_move
            new_lobos_izq = lobos_izq + lobos_move
            new_ovejas_der = ovejas_der - ovejas_move
            new_lobos_der = lobos_der - lobos_move
            new_barca_pos = 'izq'
        
        new_state = (new_ovejas_izq, new_lobos_izq, new_ovejas_der, new_lobos_der, new_barca_pos)
        
        if is_valid_state(*new_state):
            children.append(new_state)

    return children

def is_valid_state(ovejas_izq, lobos_izq, ovejas_der, lobos_der, _):
    if (ovejas_izq > 0 and ovejas_izq < lobos_izq) or (ovejas_der > 0 and ovejas_der < lobos_der):
        return False
    if ovejas_izq < 0 or lobos_izq < 0 or ovejas_der < 0 or lobos_der < 0:
        return False
    return True

def find_path_to_root(node):
    path = []
    while node:
        path.append(node.value)
        node = node.parent
    return path[::-1]

class Node:
    def __init__(self, value, parent=None):
        self.value = value
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0
    
    def add_child(self, child_state):
        child_node = Node(child_state, parent=self)
        return child_node
    
    def __lt__(self, other):
        return self.f < other.f

# A* para encontrar la solución
def construct_solution_Astar(initial_state, objective_state):
    root = Node(initial_state)  
    tree = Tree(root)  
    open_list = []  
    heapq.heappush(open_list, root)  
    visited_nodes = set()  
    visited_nodes.add(initial_state)

    operations = define_operations()  
    found = False  
    path = []  

    # Búsqueda A*
    while open_list:
        current_node = heapq.heappop(open_list)  # Obtener el nodo con el menor coste f(n)
        current_state = current_node.value

        if current_state == objective_state:  
            found = True
            path = find_path_to_root(current_node) 
            break

        children = generate_children(current_state, operations) 
        for child_state in children:
            if child_state not in visited_nodes:
                child_node = current_node.add_child(child_state)  
                child_node.g = current_node.g + 1  # El coste acumulado (g(n)) es el coste del padre + 1
                child_node.h = heuristic(child_state) 
                child_node.f = child_node.g + child_node.h  # Calcular el coste total f(n) = g(n) + h(n)
                heapq.heappush(open_list, child_node)
                visited_nodes.add(child_state)

    return tree, path, visited_nodes  

start_time = time.time()

# Ejecutar el algoritmo A*
tree, solution_path, visited_nodes = construct_solution_Astar(initial_state, objective_state)

# Mostrar la solución
print("Solución A*:", solution_path)
end_time = time.time()

print("Tiempo de ejecución:", end_time - start_time, "segundos")

Solución A*: [(2, 2, 0, 0, 'izq'), (2, 0, 0, 2, 'der'), (2, 1, 0, 1, 'izq'), (0, 1, 2, 1, 'der'), (1, 1, 1, 1, 'izq'), (0, 0, 2, 2, 'der')]
Tiempo de ejecución: 0.0009984970092773438 segundos
